# Answers to challenge questions

In [1]:
from api.models import *

## How many Starships are there?

We access our records of starships through the `Starship.objects` object.  In this case, there is a method [count](https://docs.djangoproject.com/en/1.11/ref/models/querysets/#count)

In [2]:
Starship.objects.count()

36

## How many Films did George Lucas direct?

We here, we want to use count again but this time on a subset of objects rather than all of them.  We can narrow down our query with the [filter](https://docs.djangoproject.com/en/1.11/ref/models/querysets/#filter) method by providing the desired attributes we are looking for - in this case, that 'George Lucas' is the director.


In [3]:
Film.objects.filter(director='George Lucas').count()

4

## What is the latest film George directed?

In this scenario, we want get a single film object back (or its name) instead of a number.  Our approach is the same as before to start - we filter the Films and then we order by `release_date` to get the latest one.

In [4]:
Film.objects.filter(director='George Lucas') # Returns all the films matching our criteria.

<QuerySet [<Film: A New Hope>, <Film: The Phantom Menace>, <Film: Attack of the Clones>, <Film: Revenge of the Sith>]>

In [5]:
Film.objects.filter(director='George Lucas').order_by('release_date') # orders ascending by default

<QuerySet [<Film: A New Hope>, <Film: The Phantom Menace>, <Film: Attack of the Clones>, <Film: Revenge of the Sith>]>

In [6]:
Film.objects.filter(director='George Lucas').order_by('-release_date') # -field_name orders DESCENDING

<QuerySet [<Film: Revenge of the Sith>, <Film: Attack of the Clones>, <Film: The Phantom Menace>, <Film: A New Hope>]>

In [7]:
Film.objects.filter(director='George Lucas').order_by('-release_date').first() # return the first one

<Film: Revenge of the Sith>

Django gives us the convenience methods [latest](https://docs.djangoproject.com/en/1.11/ref/models/querysets/#latest) and `earliest()` to do just this.

In [8]:
Film.objects.filter(director='George Lucas').latest('release_date')

<Film: Revenge of the Sith>

## What is the name of the Starship with id 15?

Here we can use `get` to lookup a single object, and then call its `name` attribute to access the name.

In [9]:
s = Starship.objects.get(id=15)

In [10]:
s.name

'Executor'

In [11]:
Starship.objects.get(id=15).name # same thing

'Executor'

## How many films was that starship in?

We can approach this a few ways, but we know that Films has a ManyToMany relationship with Starships, therefore we can look up across that relationsip.

In [12]:
s = Starship.objects.get(id=15)

In [13]:
s.films.all()

<QuerySet [<Film: The Empire Strikes Back>, <Film: Return of the Jedi>]>

In [14]:
s.films.count()

2

Or we can access that by filtering the Film objects themselves.

In [15]:
Film.objects.filter(starships=s)

<QuerySet [<Film: The Empire Strikes Back>, <Film: Return of the Jedi>]>

In [16]:
Film.objects.filter(starships__id=15)

<QuerySet [<Film: The Empire Strikes Back>, <Film: Return of the Jedi>]>

In [17]:
Film.objects.filter(starships__name=s.name)

<QuerySet [<Film: The Empire Strikes Back>, <Film: Return of the Jedi>]>

Either way, we just want the number.

In [18]:
Film.objects.filter(starships=s).count()

2

## How many films was the starship with the name 'CR90 corvette' in?

Same approach as before, but this time we know the name, so we can look up by that.

In [19]:
Starship.objects.get(name='CR90 corvette').films.count()

3

In [20]:
Film.objects.filter(starships__name='CR90 corvette').count()

3

## Which Film had the fewest Starships?



In [21]:
Film.objects.all()

<QuerySet [<Film: A New Hope>, <Film: The Empire Strikes Back>, <Film: Return of the Jedi>, <Film: The Phantom Menace>, <Film: Attack of the Clones>, <Film: Revenge of the Sith>]>

In [22]:
# Having python do all the work rather than Django.  This is suboptimal, but it works:

In [23]:
all_films = Film.objects.all()

In [24]:
min(all_films, key=lambda x: x.starships.count())

<Film: The Phantom Menace>

In [25]:
# Or we can annotate the count in the query itself:
from django.db.models import Count

# Here we get all the starships, give them an extra field, 
# and have python loop through and find the one with the minimum value
min(all_films.annotate(num_starships=Count('starships')), key=lambda film: film.num_starships)

<Film: The Phantom Menace>

Here we have Django do the ordering and we get the minimum value

In [26]:
Film.objects.annotate(num_starships=Count('starships')).order_by('num_starships').first()

<Film: The Phantom Menace>

In [27]:
# Same as above, just a little cleaner
Film.objects.annotate(num_starships=Count('starships')).earliest('num_starships')

<Film: The Phantom Menace>

## Which Films did Luke Skywalker (id=1) did NOT appear in?
Here we use the opposite of filter - `exclude`

In [28]:
Film.objects.exclude(characters__id=1)

<QuerySet [<Film: The Phantom Menace>, <Film: Attack of the Clones>]>

In [29]:
# Or more simply
Film.objects.exclude(characters=1)

<QuerySet [<Film: The Phantom Menace>, <Film: Attack of the Clones>]>

## How many Starships have a name with the word fight in it? (Case insensitive)

Just as we can reach across relationships and into specific fields by using `__` in our queries, we can also use it to add certain modifiers on those fields.  These are called [Field Lookups](https://docs.djangoproject.com/en/1.11/ref/models/querysets/#field-lookups) and here we'll use `icontains`, for a case insensitive string search.

In [30]:
Starship.objects.filter(name__icontains='fight')

<QuerySet [<Starship: Naboo fighter>, <Starship: Jedi starfighter>, <Starship: Belbullab-22 starfighter>]>

In [31]:
Starship.objects.filter(name__icontains='fight').count()

3